In [1]:
import tensorflow as tf
print(tf.__version__)

C:\Users\ADMIN\MEGHA\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ADMIN\MEGHA\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ADMIN\MEGHA\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ADMIN\MEGHA\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or 

1.14.0


In [2]:
!pip install nltk

In [3]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

Put the hyparameters at the top like this to make it easier to change and edit.

In [4]:
vocab_size = 5000
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_portion = .8

First, let's define two lists that containing articles and labels. In the meantime, we remove stopwords.

In [5]:
articles = []
labels = []

with open("bbc-text.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[0])
        article = row[1]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            article = article.replace(token, ' ')
            article = article.replace(' ', ' ')
        articles.append(article)
print(len(labels))
print(len(articles))

2225
2225


There are only 2,225 articles in the data. Then we split into training set and validation set, according to the parameter we set earlier, 80% for training, 20% for validation.

In [6]:
train_size = int(len(articles) * training_portion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

print(train_size)
print(len(train_articles))
print(len(train_labels))
print(len(validation_articles))
print(len(validation_labels))

1780
1780
1780
445
445


Tokenizer does all the heavy lifting for us. In our articles that it was tokenizing, it will take 5,000 most common words. oov_token is to put a special value in when an unseen word is encountered. This means I want "OOV" in bracket to be used to for words that are not in the word index. "fit_on_text" will go through all the text and create dictionary like this:

In [7]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index

You can see that "OOV" in bracket is number 1, "said" is number 2, "mr" is number 3, and so on.

In [8]:
dict(list(word_index.items())[0:10])

{'<OOV>': 1,
 'said': 2,
 'mr': 3,
 'would': 4,
 'year': 5,
 'also': 6,
 'people': 7,
 'new': 8,
 'us': 9,
 'one': 10}

This process cleans up our text, lowercase, and remove punctuations.

After tokenization, the next step is to turn thoes tokens into lists of sequence.

In [9]:
train_sequences = tokenizer.texts_to_sequences(train_articles)

This is the 11th article in the training data that has been turned into sequences.

In [10]:
print(train_sequences[10])

[2432, 1, 225, 4994, 22, 641, 586, 225, 4994, 1, 1, 1661, 1, 1, 2432, 22, 564, 1, 1, 140, 278, 1, 140, 278, 796, 822, 662, 2308, 1, 1144, 1692, 1, 1720, 4995, 1, 1, 1, 1, 1, 4737, 1, 1, 122, 4513, 1, 2, 2875, 1505, 352, 4738, 1, 52, 341, 1, 352, 2171, 3961, 41, 22, 3793, 1, 1, 1, 1, 542, 1, 1, 1, 835, 631, 2367, 347, 4739, 1, 365, 22, 1, 787, 2368, 1, 4301, 138, 10, 1, 3664, 682, 3531, 1, 22, 1, 414, 822, 662, 1, 90, 13, 633, 1, 225, 4994, 1, 599, 1, 1692, 1021, 1, 4996, 807, 1863, 117, 1, 1, 1, 2975, 22, 1, 99, 278, 1, 1607, 4997, 542, 492, 1, 1443, 4740, 778, 1320, 1, 1860, 10, 33, 641, 319, 1, 62, 478, 564, 301, 1506, 22, 479, 1, 1, 1664, 1, 797, 1, 3067, 1, 1364, 6, 1, 2432, 564, 22, 2972, 4734, 1, 1, 1, 1, 1, 850, 39, 1824, 675, 297, 26, 979, 1, 882, 22, 361, 22, 13, 301, 1506, 1342, 374, 20, 63, 883, 1096, 4302, 247]


When we train neural networks for NLP, we need sequences to be in the same size, that's why we use padding. Our max_length is 200, so we use pad_sequences to make all of our articles the same length which is 200 in my example. That's why you see that the 1st article was 426 in length, becomes 200, the 2nd article was 192 in length, becomes 200, and so on.

In [11]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [12]:
print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))

425
200
192
200
186
200


In addtion, there is padding type and truncating type, there are all "post". Means for example, for the 11th article, it was 186 in length, we padded to 200, and we padded at the end, add 14 zeros.

In [13]:
print(train_sequences[10])

[2432, 1, 225, 4994, 22, 641, 586, 225, 4994, 1, 1, 1661, 1, 1, 2432, 22, 564, 1, 1, 140, 278, 1, 140, 278, 796, 822, 662, 2308, 1, 1144, 1692, 1, 1720, 4995, 1, 1, 1, 1, 1, 4737, 1, 1, 122, 4513, 1, 2, 2875, 1505, 352, 4738, 1, 52, 341, 1, 352, 2171, 3961, 41, 22, 3793, 1, 1, 1, 1, 542, 1, 1, 1, 835, 631, 2367, 347, 4739, 1, 365, 22, 1, 787, 2368, 1, 4301, 138, 10, 1, 3664, 682, 3531, 1, 22, 1, 414, 822, 662, 1, 90, 13, 633, 1, 225, 4994, 1, 599, 1, 1692, 1021, 1, 4996, 807, 1863, 117, 1, 1, 1, 2975, 22, 1, 99, 278, 1, 1607, 4997, 542, 492, 1, 1443, 4740, 778, 1320, 1, 1860, 10, 33, 641, 319, 1, 62, 478, 564, 301, 1506, 22, 479, 1, 1, 1664, 1, 797, 1, 3067, 1, 1364, 6, 1, 2432, 564, 22, 2972, 4734, 1, 1, 1, 1, 1, 850, 39, 1824, 675, 297, 26, 979, 1, 882, 22, 361, 22, 13, 301, 1506, 1342, 374, 20, 63, 883, 1096, 4302, 247]


In [14]:
print(train_padded[10])

[2432    1  225 4994   22  641  586  225 4994    1    1 1661    1    1
 2432   22  564    1    1  140  278    1  140  278  796  822  662 2308
    1 1144 1692    1 1720 4995    1    1    1    1    1 4737    1    1
  122 4513    1    2 2875 1505  352 4738    1   52  341    1  352 2171
 3961   41   22 3793    1    1    1    1  542    1    1    1  835  631
 2367  347 4739    1  365   22    1  787 2368    1 4301  138   10    1
 3664  682 3531    1   22    1  414  822  662    1   90   13  633    1
  225 4994    1  599    1 1692 1021    1 4996  807 1863  117    1    1
    1 2975   22    1   99  278    1 1607 4997  542  492    1 1443 4740
  778 1320    1 1860   10   33  641  319    1   62  478  564  301 1506
   22  479    1    1 1664    1  797    1 3067    1 1364    6    1 2432
  564   22 2972 4734    1    1    1    1    1  850   39 1824  675  297
   26  979    1  882   22  361   22   13  301 1506 1342  374   20   63
  883 1096 4302  247    0    0    0    0    0    0    0    0    0    0
    0 

And for the 1st article, it was 426 in length, we truncated to 200, and we truncated at the end.

In [15]:
print(train_sequences[0])

[91, 160, 1141, 1106, 49, 979, 755, 1, 89, 1304, 4288, 129, 175, 3653, 1213, 1194, 1577, 42, 7, 893, 91, 1, 334, 85, 20, 14, 130, 3262, 1214, 2422, 569, 451, 1375, 58, 3378, 3521, 1659, 8, 921, 730, 10, 844, 1, 9, 597, 1578, 1107, 395, 1941, 1106, 731, 49, 537, 1397, 2012, 1622, 134, 249, 113, 2356, 795, 4979, 980, 583, 10, 3955, 3956, 921, 2563, 129, 344, 175, 3653, 1, 1, 39, 62, 2868, 28, 9, 4722, 18, 1305, 136, 416, 7, 143, 1422, 71, 4500, 436, 4980, 91, 1107, 77, 1, 82, 2013, 53, 1, 91, 6, 1008, 598, 89, 1304, 91, 1964, 131, 137, 420, 9, 2869, 38, 152, 1234, 89, 1304, 4723, 7, 436, 4980, 3154, 6, 2493, 1, 431, 1126, 1, 1423, 570, 1215, 1903, 1, 766, 9, 537, 1397, 2012, 134, 2069, 400, 845, 1965, 1600, 34, 1716, 2870, 1, 1, 2423, 244, 9, 2625, 82, 732, 6, 1172, 1195, 152, 720, 590, 1, 124, 28, 1305, 1688, 432, 83, 933, 115, 20, 14, 18, 3155, 1, 37, 1483, 1, 23, 37, 87, 335, 2357, 37, 467, 255, 1965, 1358, 328, 1, 299, 732, 1173, 18, 2871, 1716, 1, 294, 756, 1074, 395, 2014, 387, 431

In [16]:
print(train_padded[0])

[  91  160 1141 1106   49  979  755    1   89 1304 4288  129  175 3653
 1213 1194 1577   42    7  893   91    1  334   85   20   14  130 3262
 1214 2422  569  451 1375   58 3378 3521 1659    8  921  730   10  844
    1    9  597 1578 1107  395 1941 1106  731   49  537 1397 2012 1622
  134  249  113 2356  795 4979  980  583   10 3955 3956  921 2563  129
  344  175 3653    1    1   39   62 2868   28    9 4722   18 1305  136
  416    7  143 1422   71 4500  436 4980   91 1107   77    1   82 2013
   53    1   91    6 1008  598   89 1304   91 1964  131  137  420    9
 2869   38  152 1234   89 1304 4723    7  436 4980 3154    6 2493    1
  431 1126    1 1423  570 1215 1903    1  766    9  537 1397 2012  134
 2069  400  845 1965 1600   34 1716 2870    1    1 2423  244    9 2625
   82  732    6 1172 1195  152  720  590    1  124   28 1305 1688  432
   83  933  115   20   14   18 3155    1   37 1483    1   23   37   87
  335 2357   37  467  255 1965 1358  328    1  299  732 1173   18 2871
 1716 

Then we do the same for the validation sequences. Note that we should expect more out of vocabulary words from validation articles because word index were derived from the training articles.

In [17]:
validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(validation_sequences))
print(validation_padded.shape)

445
(445, 200)


Now we are going to look at the labels. because our labels are text, so we will tokenize them, when training, labels are expected to be numpy arrays. So we will turn list of labels into numpy arrays like so:

In [18]:
print(set(labels))

{'entertainment', 'politics', 'sport', 'business', 'tech'}


In [19]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

In [20]:
print(training_label_seq[0])
print(training_label_seq[1])
print(training_label_seq[2])
print(training_label_seq.shape)

print(validation_label_seq[0])
print(validation_label_seq[1])
print(validation_label_seq[2])
print(validation_label_seq.shape)

[4]
[2]
[1]
(1780, 1)
[5]
[4]
[3]
(445, 1)


Before training deep neural network, we want to explore what our original article and article after padding look like. Running the following code, we explore the 11th article, we can see that some words become "OOV", because they did not make to the top 5,000.

In [21]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_article(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])
print(decode_article(train_padded[10]))
print('---')
print(train_articles[10])

berlin <OOV> anti nazi film german movie anti nazi <OOV> <OOV> drawn <OOV> <OOV> berlin film festival <OOV> <OOV> final days <OOV> final days member white rose movement <OOV> 21 arrested <OOV> brother hans <OOV> <OOV> <OOV> <OOV> <OOV> tyranny <OOV> <OOV> director marc <OOV> said feeling responsibility keep legacy <OOV> going must <OOV> keep ideas alive added film drew <OOV> <OOV> <OOV> <OOV> trial <OOV> <OOV> <OOV> east germany secret police discovery <OOV> behind film <OOV> worked closely <OOV> relatives including one <OOV> sisters ensure historical <OOV> film <OOV> members white rose <OOV> group first started <OOV> anti nazi <OOV> summer <OOV> arrested dropped <OOV> munich university calling day <OOV> <OOV> <OOV> regime film <OOV> six days <OOV> arrest intense trial saw <OOV> initially deny charges ended <OOV> appearance one three german films <OOV> top prize festival south african film version <OOV> <OOV> opera <OOV> shot <OOV> town <OOV> language also <OOV> berlin festival film en

Now we can implement LSTM. Here is my code that I build a tf.keras.Sequential model and start with an embedding layer. An embedding layer stores one vector per word. When called, it converts the sequences of word indices into sequences of vectors. After training, words with similar meanings often have the similar vectors.

Next is how to implement LSTM in code. The Bidirectional wrapper is used with a LSTM layer, this propagates the input forwards and backwards through the LSTM layer and then concatenates the outputs. This helps LSTM to learn long term dependencies. We then fit it to a dense neural network to do classification.

This index-lookup is much more efficient than the equivalent operation of passing a one-hot encoded vector through a tf.keras.layers.Dense layer.

In [22]:
model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # use ReLU in place of tanh function since they are very good alternatives of each other.
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    # Add a Dense layer with 6 units and softmax activation.
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(6, activation='softmax')
])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               66048     
____________________________________________________________

In our model summay, we have our embeddings, our Bidirectional contains LSTM, followed by two dense layers. The output from Bidirectional is 128, because it doubled what we put in LSTM. We can also stack LSTM layer but I found the results worse.

In [23]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [24]:
num_epochs = 10
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), verbose=2)

Train on 1780 samples, validate on 445 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
1780/1780 - 48s - loss: 1.5767 - acc: 0.3174 - val_loss: 1.1998 - val_acc: 0.5978
Epoch 2/10
1780/1780 - 33s - loss: 0.8689 - acc: 0.6933 - val_loss: 0.5627 - val_acc: 0.8517
Epoch 3/10
1780/1780 - 79s - loss: 0.2649 - acc: 0.9270 - val_loss: 0.4415 - val_acc: 0.8449
Epoch 4/10
1780/1780 - 35s - loss: 0.1596 - acc: 0.9461 - val_loss: 0.3090 - val_acc: 0.8876
Epoch 5/10
1780/1780 - 42s - loss: 0.1004 - acc: 0.9624 - val_loss: 0.3489 - val_acc: 0.8966
Epoch 6/10
1780/1780 - 73s - loss: 0.0811 - acc: 0.9730 - val_loss: 0.2810 - val_acc: 0.9191
Epoch 7/10
1780/1780 - 39s - loss: 0.0421 - acc: 0.9871 - val_loss: 0.2715 - val_acc: 0.9213
Epoch 8/10
1780/1780 - 68s - loss: 0.0203 - acc: 0.9955 - val_loss: 0.2753 - val_acc: 0.9258
Epoch 9/10
1780/1780 - 46s - loss: 0.0169 - acc: 0.9949 - val_loss: 0.3926 - val_acc: 0.8966
Epoch 10/10
1780/1780 

In [25]:
!pip install matplotlib

In [27]:
txt = ["A WeWork shareholder has taken the company to court over the near-$1.7bn (£1.3bn) leaving package approved for ousted co-founder Adam Neumann."]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_length)
pred = model.predict(padded)
pred

array([[1.5710984e-05, 9.8717695e-01, 9.7253080e-04, 3.6261771e-03,
        1.7445066e-04, 8.0341110e-03]], dtype=float32)

In [28]:
txt = ["A WeWork shareholder has taken the company to court over the near-$1.7bn (£1.3bn) leaving package approved for ousted co-founder Adam Neumann."]
seq = tokenizer.texts_to_sequences(txt)
padded = pad_sequences(seq, maxlen=max_length)
pred = model.predict(padded)
labels = ['sport', 'bussiness', 'politics', 'tech', 'entertainment','unknown']
print(pred, labels[np.argmax(pred)])

[[1.5710984e-05 9.8717695e-01 9.7253080e-04 3.6261771e-03 1.7445066e-04
  8.0341110e-03]] bussiness


In [29]:
np.argmax(pred)

1